

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [1]:
# %%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-m82vg0yq/unsloth_bf5f6c8073d9416daef6fbaaf51ae17d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-m82vg0yq/unsloth_bf5f6c8073d9416daef6fbaaf51ae17d
  Resolved https://github.com/unslothai/unsloth.git to commit f2b304bd69ddb034fd34e5d663319c0d2fbe4ae4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.2/272.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 148.4 MB/s eta 0:00:00

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [3]:
import re

def clean_text(x):
    def _clean_one(text):
        if isinstance(text, list):
            text = " ".join(str(t) for t in text)

        text = str(text)
        text = re.sub(r"```.*?```", "", text, flags=re.DOTALL)
        text = re.sub(r"`([^`]*)`", r"\1", text)
        text = re.sub(r"(?<=\d),(?=\d)", "", text)
        text = re.sub(r"[０-９]", lambda m: chr(ord(m.group(0)) - 65248), text)
        text = re.sub(r"\s+", " ", text).strip()

        return text

    if isinstance(x, list):
        return [_clean_one(item) for item in x]
    else:
        return _clean_one(x)


In [4]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(10000))      # Use the first 5,000 for training
validation_dataset = shuffled_dataset.select(range(10000, 10500)) # Use the next 500 for validation

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
# The instructional prompt template for training
training_prompt = best_prompt_candidate = """You are a great mathematician. You must decide if the Solution to the Question is correct.
Your response should be exactly 'True' if the Solution's final answer is correct.
Your response should be exactly 'False' if the Solution's final answer is not correct.
Question:
{}
Solution:
{}
Output:
{}"""




# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = clean_text(examples["question"])
    solutions = clean_text(examples["solution"])
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", "lm_head"],
    lora_alpha = 64, # A common practice is to set alpha = 2 * r
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.10.11 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training lm_head in mixed precision to save VRAM



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 250,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 10,000 | Num Epochs = 4 | Total steps = 250
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 4 x 1) = 128
 "-____-"     Trainable parameters = 609,222,656 of 8,639,483,904 (7.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.279400
20,0.790700
30,0.728600
40,0.712100
50,0.704200
60,0.706600
70,0.700700
80,0.684900
90,0.658600
100,0.654000


Step,Training Loss
10,1.279400
20,0.790700
30,0.728600
40,0.712100
50,0.704200
60,0.706600
70,0.700700
80,0.684900
90,0.658600
100,0.654000


TrainOutput(global_step=250, training_loss=0.680504415512085, metrics={'train_runtime': 6309.5154, 'train_samples_per_second': 5.072, 'train_steps_per_second': 0.04, 'total_flos': 1.0517376009653453e+18, 'train_loss': 0.680504415512085, 'epoch': 3.1661341853035143})


## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [9]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = best_prompt_candidate = """You are a great mathematician. You must decide if the Solution to the Question is correct.
Your response should be exactly 'True' if the Solution's final answer is correct.
Your response should be exactly 'False' if the Solution's final answer is not correct.
Question:
{}
Solution:
{}
Output:
"""




# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = clean_text(example["question"])
solution = clean_text(example["solution"])

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True, do_sample=False, temperature=0.0)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
A circular spinner for a game has a radius of 10 cm. The probability of winning on one spin of this spinner is $\frac{2}{5}$. What is the area, in sq cm, of the WIN sector? Express your answer in terms of $\pi$. [asy]import graph; draw(Circle((00),25),black); draw((00)--(718),Arrow); draw((00)--(025)); draw((00)--(15,-20)); label("WIN",(1010),S); label("LOSE",(-8,-8),N); dot((00)); [/asy]

#### SOLUTION ####
Let $A$ be the area of the win sector, $C$ be the area of the entire circle, and $P$ be the probability of winning on one spin. The probability of winning on one spin is equal to the ratio of the area of the win sector to the area of the entire circle: \[ P = \frac{A}{C} = \frac{A}{\pi \times 10^2} \] From this equation, we can solve for $A$ by multiplying both sides by $\pi \times 10^2$: \[ A = P \times \pi \times 10^2 = \boxed{N} \] Thus, the area of the win sector is $N$ square centimeters.

#### MODEL'S PREDICTION ####
False<|end_of_text|>

#### CORRECT ANSWE

In [10]:
print("\n===== Calculating Validation Accuracy =====")

correct = 0
total = 0

for i, example in enumerate(validation_dataset):
    question = clean_text(example["question"])
    solution = clean_text(example["solution"])
    true_label = str(example["is_correct"]).strip().lower()

    inputs = tokenizer(
        [inference_prompt.format(question, str(solution))],
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True, do_sample=False, temperature=0.0)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    if "Output:" in response:
        pred_text = response.split("Output:")[-1].strip()
    else:
        pred_text = response.strip()

    pred_label = "true" if "true" in pred_text.lower() else "false"

    if pred_label == true_label:
        correct += 1
    total += 1

    if i % 25 == 0 and i > 0:
        print(f"Processed {i}/{len(validation_dataset)} examples...")

val_accuracy = 100 * correct / total
print(f"\nValidation Accuracy: {val_accuracy:.2f}%  ({correct}/{total})")



===== Calculating Validation Accuracy =====
Processed 25/500 examples...
Processed 50/500 examples...
Processed 75/500 examples...
Processed 100/500 examples...
Processed 125/500 examples...
Processed 150/500 examples...
Processed 175/500 examples...
Processed 200/500 examples...
Processed 225/500 examples...
Processed 250/500 examples...
Processed 275/500 examples...
Processed 300/500 examples...
Processed 325/500 examples...
Processed 350/500 examples...
Processed 375/500 examples...
Processed 400/500 examples...
Processed 425/500 examples...
Processed 450/500 examples...
Processed 475/500 examples...

Validation Accuracy: 80.80%  (404/500)


## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [12]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = clean_text(example["question"])
    solution = clean_text(example["solution"])

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True, do_sample=False, temperature=0.0)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 23%|██▎       | 2329/10000 [10:43<35:11,  3.63it/s]Unsloth: Input IDs of shape torch.Size([1, 1025]) with length 1025 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 30%|███       | 3050/10000 [14:03<31:56,  3.63it/s]Unsloth: Input IDs of shape torch.Size([1, 1184]) with length 1184 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 33%|███▎      | 3251/10000 [14:59<30:49,  3.65it/s]Unsloth: Input IDs of shape torch.Size([1, 1042]) with length 1042 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 35%|███▌      | 3547/10000 [16:21<29:11,  3.68it/s]Unsloth: Input IDs of shape torch.Size([1, 1249]) with length 1249 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 54%|█████▍    | 5412/10000 


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [14]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/llama3_8b_math_verifier_checkpoint


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a world-class amazing mathematician and logic verifier.
Your task is to *carefully* evaluate whether the given "Solution" correctly and completely solves the "Question".
Think step by step, check each reasoning step and final result precisely.
If the solution is mathematically valid and all reasoning steps are logically sound, output exactly "True".
Otherwise, if there is *any* mathematical, logical, or procedural mistake, output exactly "False".
Do not include explanations or extra text.

Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True, do_sample=False, temperature=0.0)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")